In [56]:
from datasets import load_dataset
import os
import numpy as np
import torch
from scipy.special import softmax

In [57]:
from transformers import DistilBertTokenizerFast
#tokenizer = DistilBertTokenizerFast.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [62]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [58]:
def encode(examples):
     return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

In [86]:
# training model on tokenized and split data
class Dataset(torch.utils.data.Dataset):
    def __init__(self, inputs):
        self.inputs = inputs

    def __getitem__(self, idx):
        item = {key: torch.tensor(val) for key, val in self.inputs[idx].items() if key != 'text'}
        return item

    def __len__(self):
        return len(self.inputs)

In [87]:
def prepare_data(filePath):
    # loading features and labels per patient
    input_dataset = load_dataset('text', data_files={'test': filePath})
    
    # applying encoding function to dataset
    input_dataset = input_dataset.map(encode, batched=True)
    
    # initiating dataset object
    test_dataset = Dataset(input_dataset['test'])

    return test_dataset

In [63]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

trainer = Trainer(
    model=model                        # the instantiated 🤗 Transformers model to be trained
)

In [85]:
# load the patient datset with file path
test_dataset = prepare_data('../../data/processPatient/5087')

# generate probability of readmission
pred = trainer.predict(test_dataset).predictions

# softmax each row so each row sums to 1
prob = softmax(pred, axis = 1)

# write the probabilities out to text file
with open('../../data/probability/5087','w') as f:
    for row in prob:
        for column in row:
            f.write(str(column))
            f.write(" ")
        f.write('\n')

Using custom data configuration default
Reusing dataset text (/home/ubuntu/.cache/huggingface/datasets/text/default-a0590ff645070a10/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab)
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-a0590ff645070a10/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab/cache-6c463743f912fad4.arrow
